# 3 Import Libraries & Data

In [1]:
# Import libraries, visualization
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Set path
path = r"C:\Users\miche\Instacart Basket Analysis 2022_MP\02 Data"

In [3]:
# Import customers.csv
customer=pd.read_csv(os.path.join(path, 'Original Data', 'customers.csv'), index_col = False)

In [4]:
# Import most recent pkl
flags = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_flags.pkl'))

In [5]:
# Explore Data
customer.shape

(206209, 10)

In [6]:
customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [7]:
customer.columns

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [8]:
customer.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [9]:
customer.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [10]:
flags.columns

Index(['order_id', 'customer_id', 'order_number', 'order_day', 'order_hour',
       'previous_order', 'product_id', 'add_to_cart_order', 'reordered',
       'product_name', 'department_id', 'prices', '_merge', 'price_range',
       'busy_day', 'busy_days', 'busy_hours', 'max_order', 'loyalty_flag',
       'average_price', 'spender_flag', 'median_previous_order',
       'customer_frequency'],
      dtype='object')

In [11]:
flags.dtypes

order_id                    int64
customer_id                 int64
order_number                int64
order_day                   int64
order_hour                  int64
previous_order            float64
product_id                  int64
add_to_cart_order           int64
reordered                   int64
product_name               object
department_id               int64
prices                    float64
_merge                   category
price_range                object
busy_day                   object
busy_days                  object
busy_hours                 object
max_order                   int64
loyalty_flag               object
average_price             float64
spender_flag               object
median_previous_order     float64
customer_frequency         object
dtype: object

# 4 Wrangle Customer Data

In [12]:
# Rename Columns
customer.rename(columns = {'user_id' : 'customer_id'}, inplace = True)
customer.rename(columns = {'n_dependants' : 'dependents'}, inplace = True)
customer.rename(columns = {'Gender' : 'gender'}, inplace = True)
customer.rename(columns = {'STATE' : 'state'}, inplace = True)
customer.rename(columns = {'Age' : 'age'}, inplace = True)

In [13]:
customer.columns

Index(['customer_id', 'First Name', 'Surnam', 'gender', 'state', 'age',
       'date_joined', 'dependents', 'fam_status', 'income'],
      dtype='object')

In [14]:
# Drop Columns
customer = customer.drop(columns = ['First Name', 'Surnam'])

In [15]:
customer.columns

Index(['customer_id', 'gender', 'state', 'age', 'date_joined', 'dependents',
       'fam_status', 'income'],
      dtype='object')

In [16]:
customer.shape

(206209, 8)

# 5 Data Consistency Checks

In [17]:
# Check for mixed type data
for col in customer.columns.tolist():
  weird = (customer[[col]].applymap(type) != customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customer[weird]) > 0:
    print (col)

In [18]:
# Check for missing values
customer.isnull().sum()

customer_id    0
gender         0
state          0
age            0
date_joined    0
dependents     0
fam_status     0
income         0
dtype: int64

In [19]:
# Find duplicates
cust_dups = customer[customer.duplicated()]

In [20]:
cust_dups

,customer_id,gender,state,age,date_joined,dependents,fam_status,income


#### Customers.csv returned no mixed data types, no missing values, no duplicates. 

In [21]:
customer.shape

(206209, 8)

In [23]:
# Drop Column in df_flags
flags = flags.drop(columns = ['_merge'])

In [25]:
flags.columns

Index(['order_id', 'customer_id', 'order_number', 'order_day', 'order_hour',
       'previous_order', 'product_id', 'add_to_cart_order', 'reordered',
       'product_name', 'department_id', 'prices', 'price_range', 'busy_day',
       'busy_days', 'busy_hours', 'max_order', 'loyalty_flag', 'average_price',
       'spender_flag', 'median_previous_order', 'customer_frequency'],
      dtype='object')

# 6 Combine customer.csv with most recent dataset

In [26]:
# Test merge without overwriting 
pd.merge(flags,customer, on = ['customer_id'], indicator = True)

,order_id,customer_id,order_number,order_day,order_hour,previous_order,product_id,add_to_cart_order,reordered,product_name,...,median_previous_order,customer_frequency,gender,state,age,date_joined,dependents,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,...,21.0,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,12427,3,1,Original Beef Jerky,...,21.0,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,3367565,1,6,2,7,19.0,12427,2,1,Original Beef Jerky,...,21.0,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2550362,1,10,4,8,30.0,12427,9,1,Original Beef Jerky,...,21.0,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,473747,1,3,3,12,21.0,196,1,1,Soda,...,21.0,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6481350,2654287,114962,4,3,15,14.0,38449,5,0,Ocean Mist Pods,...,14.0,Regular customer,Male,Delaware,36,8/24/2019,0,single,53726,both
6481351,1188714,131830,1,4,11,NaN,5847,2,0,Ultra Sensitive Hydra Gel,...,NaN,NaN,Female,Vermont,50,1/10/2020,3,married,53427,both
6481352,2714916,193614,2,6,18,6.0,22554,3,0,Sugar Free Low Calorie Artificially Flavored B...,...,5.0,Frequent customer,Male,Kentucky,69,7/21/2019,3,married,119757,both
6481353,2329696,193614,3,3,17,4.0,3249,3,1,Medium Thick and Chunky Fiesta Salsa,...,5.0,Frequent customer,Male,Kentucky,69,7/21/2019,3,married,119757,both


In [28]:
# Perform merge
merged = flags.merge(customer, on = ['customer_id'], indicator = True)

In [29]:
# Explore new dataset
merged.shape

(6481355, 30)

In [30]:
merged.columns

Index(['order_id', 'customer_id', 'order_number', 'order_day', 'order_hour',
       'previous_order', 'product_id', 'add_to_cart_order', 'reordered',
       'product_name', 'department_id', 'prices', 'price_range', 'busy_day',
       'busy_days', 'busy_hours', 'max_order', 'loyalty_flag', 'average_price',
       'spender_flag', 'median_previous_order', 'customer_frequency', 'gender',
       'state', 'age', 'date_joined', 'dependents', 'fam_status', 'income',
       '_merge'],
      dtype='object')

In [32]:
merged.head()

,order_id,customer_id,order_number,order_day,order_hour,previous_order,product_id,add_to_cart_order,reordered,product_name,...,median_previous_order,customer_frequency,gender,state,age,date_joined,dependents,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,...,21.0,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,12427,3,1,Original Beef Jerky,...,21.0,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,3367565,1,6,2,7,19.0,12427,2,1,Original Beef Jerky,...,21.0,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2550362,1,10,4,8,30.0,12427,9,1,Original Beef Jerky,...,21.0,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,473747,1,3,3,12,21.0,196,1,1,Soda,...,21.0,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both


In [33]:
merged.describe()

,order_id,customer_id,order_number,order_day,order_hour,previous_order,product_id,add_to_cart_order,reordered,department_id,prices,max_order,average_price,median_previous_order,age,dependents,income
count,6.481355e+06,6.481355e+06,6.481355e+06,6.481355e+06,6.481355e+06,6.065562e+06,6.481355e+06,6.481355e+06,6.481355e+06,6.481355e+06,6.481355e+06,6.481355e+06,6.481355e+06,6.478316e+06,6.481355e+06,6.481355e+06,6.481355e+06
mean,1.710967e+06,1.029364e+05,1.714780e+01,2.739231e+00,1.342433e+01,1.110206e+01,2.559877e+04,8.349040e+00,5.895449e-01,9.919076e+00,1.216758e+01,3.287300e+01,1.216758e+01,1.041821e+01,4.945049e+01,1.502168e+00,9.940574e+04
std,9.873891e+05,5.946519e+04,1.754617e+01,2.090380e+00,4.246312e+00,8.777455e+00,1.408289e+04,7.126729e+00,4.919164e-01,6.280990e+00,5.081096e+02,2.514877e+01,1.235353e+02,7.199974e+00,1.848479e+01,1.118759e+00,4.305555e+04
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,2.590300e+04
25%,8.563190e+05,5.142900e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.354100e+04,3.000000e+00,0.000000e+00,4.000000e+00,4.200000e+00,1.300000e+01,7.246734e+00,6.000000e+00,3.300000e+01,1.000000e+00,6.698400e+04
50%,1.711571e+06,1.026190e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,2.527900e+04,6.000000e+00,1.000000e+00,9.000000e+00,7.400000e+00,2.600000e+01,7.800000e+00,8.000000e+00,4.900000e+01,2.000000e+00,9.657800e+04
75%,2.566164e+06,1.543820e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01,3.794700e+04,1.100000e+01,1.000000e+00,1.600000e+01,1.130000e+01,4.700000e+01,8.363033e+00,1.300000e+01,6.500000e+01,3.000000e+00,1.278310e+05
max,3.421083e+06,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,1.440000e+02,1.000000e+00,2.100000e+01,9.999900e+04,9.900000e+01,3.333710e+04,3.000000e+01,8.100000e+01,3.000000e+00,5.939010e+05


In [35]:
# Check mixed-type data
for col in merged.columns.tolist():
  weird = (merged[[col]].applymap(type) != merged[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (merged[weird]) > 0:
    print (col)

customer_frequency


In [36]:
merged['customer_frequency'].dtype

dtype('O')

In [37]:
# Export pkl 
merged.to_pickle(os.path.join(path,'Prepared Data', 'orders_products_customers_merged.pkl'))